In [1]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
  Using cached matplotlib-3.5.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


'\nRemember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!\nAlternatively, you can uncomment the exit() below to crash and restart the kernel, in the case\nthat you want to use the "Run All Cells" (or similar) option.\n'

In [2]:
# exit()

In [2]:
# Папка, где будет размещаться датасет Golos
data_dir = '/content'

In [3]:
! ls /content

sample_data


# Dataset load

In [4]:
import glob
import os
import subprocess
import tarfile
import wget

# Загрузка датасета Golos
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

def extract_tar(tar_path, dir_name ):
  if not os.path.exists(os.path.join(data_dir, dir_name)):
    tar = tarfile.open(tar_path)
    tar.extractall(path=data_dir)

golos_path = load("https://sc.link/Kqr", "test.tar")
golos_path_train = load("https://sc.link/1Z3", "train.tar")

extract_tar(golos_path, '/test/')
extract_tar(golos_path_train, '/train/')

Dataset downloaded at: /content/test.tar
Dataset downloaded at: /content/train.tar


In [18]:
!wget https://github.com/karpnv/nis_speech/raw/master/train_manifest.jsonl.zip
!unzip /content/train_manifest.jsonl.zip

--2021-11-30 05:19:37--  https://github.com/karpnv/nis_speech/raw/master/train_manifest.jsonl.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/karpnv/nis_speech/master/train_manifest.jsonl.zip [following]
--2021-11-30 05:19:37--  https://raw.githubusercontent.com/karpnv/nis_speech/master/train_manifest.jsonl.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52145788 (50M) [application/zip]
Saving to: ‘train_manifest.jsonl.zip.1’

train_manifest.json 100%[===================>]  49.73M   153MB/s    in 0.3s    

2021-11-30 05:19:37 (153 MB/s) - ‘train_manifest.jsonl.zip.1’ saved [52145788/5214

In [19]:
!ls -la /content

total 17104320
drwxr-xr-x 1 root root        4096 Nov 30 05:19 .
drwxr-xr-x 1 root root        4096 Nov 30 04:53 ..
drwxr-xr-x 4 root root        4096 Nov 18 14:35 .config
-rw-r--r-- 1 root root    71179174 Nov 30 05:18 QuartzNet15x5_golos.nemo
drwxr-xr-x 1 root root        4096 Nov 18 14:36 sample_data
drwxr-xr-x 4 1000 1000        4096 Apr 20  2021 test
-rw-r--r-- 1 root root  1628508160 Nov 30 04:59 test.tar
drwxr-xr-x 3 root root        4096 Nov 30 05:17 train
-rw-r--r-- 1 root root   350713145 Apr 20  2021 train_manifest.jsonl
-rw-r--r-- 1 root root    52145788 Nov 30 05:17 train_manifest.jsonl.zip
-rw-r--r-- 1 root root    52145788 Nov 30 05:19 train_manifest.jsonl.zip.1
-rw-r--r-- 1 root root 15360092160 Nov 30 05:11 train.tar


In [20]:
!ls -la /content/test/farfield

total 1280
drwxr-xr-x 3 1000 1000   4096 Nov 30 05:17 .
drwxr-xr-x 4 1000 1000   4096 Apr 20  2021 ..
-rw-r--r-- 1 root root 605893 Nov 30 05:17 farfield.jsonl
drwxr-xr-x 2 1000 1000 131072 Mar 14  2021 files
-rw-r--r-- 1 1000 1000 561825 Mar 17  2021 manifest.jsonl


In [21]:
!ls -la /content/train/ | head

total 393360
drwxr-xr-x 3 root root      4096 Nov 30 05:17 .
drwxr-xr-x 1 root root      4096 Nov 30 05:19 ..
drwxr-xr-x 2 1000 1000   8052736 Mar 14  2021 farfield
-rw-r--r-- 1 root root  44018241 Nov 30 05:18 manifest.jsonl
-rw-r--r-- 1 root root 350713145 Apr 20  2021 train_manifest.jsonl


In [9]:
!mv /content/train_manifest.jsonl /content/train/

# Prepare data

In [22]:
import json

def build_manifest(manifest_rel, manifest_abs, parse_farfield=True, test_data=True):
  manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
  with open(manifest_rel, 'r') as fin:
    with open(manifest_abs, 'w') as fout:
      for line in fin:
        metadata = json.loads(line)
        if metadata['audio_filepath'].startswith('farfield') or not parse_farfield:
          if metadata['duration'] > 0.1 or test_data:
            metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
            json.dump(metadata, fout)
            fout.write('\n')

def build_manifest_filtered(manifest_rel, manifest_abs, filter_id,parse_farfield=True, test_data=True, rand_ratio=1):
  manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
  with open(manifest_rel, 'r') as fin:
    with open(manifest_abs, 'w') as fout:
      for line in fin:
        metadata = json.loads(line)
        if metadata['audio_filepath'].startswith('farfield') or not parse_farfield:
          if (metadata['duration'] > 0.1 or test_data) and metadata['id'] in filter_id:
            metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
            json.dump(metadata, fout)
            fout.write('\n')

In [23]:
print("******")
train_rel = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
train_abs = os.path.join(data_dir, 'test/farfield/farfield.jsonl')
# if not os.path.isfile(train_abs):
build_manifest(train_rel, train_abs, parse_farfield=False)
test_manifest = train_abs
print("test_manifest", test_manifest)

train_rel = os.path.join(data_dir, 'train/train_manifest.jsonl')
train_abs = os.path.join(data_dir, 'train/manifest.jsonl')
# if not os.path.isfile(train_abs):
build_manifest(train_rel, train_abs, test_data=False)
train_manifest = train_abs
print("train_manifest", train_manifest)

******
test_manifest /content/test/farfield/farfield.jsonl
train_manifest /content/train/manifest.jsonl


In [12]:
!ls -la test/farfield/manifest.jsonl

-rw-r--r-- 1 1000 1000 561825 Mar 17  2021 test/farfield/manifest.jsonl


In [13]:
test_manifest, train_manifest

('/content/test/farfield/farfield.jsonl', '/content/train/manifest.jsonl')

In [14]:
! head /content/test/farfield/farfield.jsonl

{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "/content/test/farfield/files/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}
{"id": "26093ef7a8c5ec6a3c586a6a929c1bd3", "audio_filepath": "/content/test/farfield/files/26093ef7a8c5ec6a3c586a6a929c1bd3.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0432\u044b\u0437\u043e\u0432 \u0441\u0432\u0435\u0442\u043b\u0430\u043d\u0435 \u0432\u0430\u0441\u0438\u043b\u044c\u0435\u0432\u043d\u0435 \u043d\u0438\u043a\u043e\u043b\u0435\u043d\u043a\u043e", "duration": 3.1720625}
{"id": "b42d13ab3dc159fac39a1120e8011f66", "audio_filepath": "/content/test/farfield/files/b42d13ab3dc159fac39a1120e8011f66.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.5226875}
{"id": "7e55843e745a239b35212e2caba77239", "audio_filepath": "/content/test/farfield/files/7e55843e745a239b35212e2caba77239.wav", "text": "\u0434\u0436\

In [15]:
! head /content/train/manifest.jsonl

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}
{"id": "f6075fae0c592183f107cf781117d9b5", "audio_filepath": "/content/train/farfield/f6075fae0c592183f107cf781117d9b5.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u043e\u0447\u0443 \u043f\u043e\u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0441 \u0434\u0436\u043e\u0439", "duration": 4.15125}
{"id": "2599a6cb55795b38ddd80b849d8670bc", "audio_filepath": "/content/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav", "text": "\u0434\u0436\u043e\u0439 \u043a\u0430\u043a\u0430\u044f \u0432\u0430\u043b\u044e\u0442\u0430 \u0432 \u043a\u0430\u043c\u0431\u043e\u0434\u0436\u0435", "duration": 3.9185625}

# Обучение Модели

## Загрузка предобученной модели

In [16]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-30 05:18:20 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [17]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

Dataset downloaded at: /content/QuartzNet15x5_golos.nemo


[NeMo W 2021-11-30 05:18:32 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-30 05:18:32 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-30 05:18:32 features:265] PADDING: 16
[NeMo I 2021-11-30 05:18:32 features:282] STFT using torch
[NeMo I 2021-11-30 05:18:50 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


## Получение данных о записях

In [24]:
files = []
texts = []
ids = []
with open(train_manifest, 'r') as fin:
  for line in fin:
    metadata = json.loads(line)
    files.append(metadata['audio_filepath'])
    texts.append(metadata['text'])
    ids.append(metadata['id'])
# files = [data_dir + "/test/crowd/files/e632f7d39c15e7edfc665b91e6f2071f.wav",
#          data_dir + "/test/crowd/files/5db5df8bb9e3b6660b2a04b34d4a355d.wav"]

In [25]:
files[:5]

['/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav',
 '/content/train/farfield/f6075fae0c592183f107cf781117d9b5.wav',
 '/content/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav',
 '/content/train/farfield/5f7483035b2f26b773b0d3fb125b779d.wav',
 '/content/train/farfield/4c95a12b71e1262ac937d2fe6a803906.wav']

In [26]:
texts[:5]

['афина хотела номер телефона поменять карты',
 'салют хочу поговорить с джой',
 'джой какая валюта в камбодже',
 'салют я могу что нибудь сделать',
 'джой смотреть операцию ы и другие приключения шурика онлайн']

In [27]:
import librosa
import IPython.display as ipd

# Load and listen to the audio file
example_file = '/content/train/farfield/4c95a12b71e1262ac937d2fe6a803906.wav'

audio, sample_rate = librosa.load(example_file)
ipd.Audio(example_file, rate=sample_rate)

## Получение предсказаний

In [28]:
transcriptions = asr_model.transcribe(paths2audio_files=files)

Transcribing:   0%|          | 0/31000 [00:00<?, ?it/s]

[NeMo W 2021-11-30 05:22:40 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-30 05:22:41 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


In [29]:
files[:10]

['/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav',
 '/content/train/farfield/f6075fae0c592183f107cf781117d9b5.wav',
 '/content/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav',
 '/content/train/farfield/5f7483035b2f26b773b0d3fb125b779d.wav',
 '/content/train/farfield/4c95a12b71e1262ac937d2fe6a803906.wav',
 '/content/train/farfield/90655e9a042c4e79c424cd7dd82b37a6.wav',
 '/content/train/farfield/14b5e99616d446c3887c8927403632a0.wav',
 '/content/train/farfield/d71107319855ef21249be2ab0edea9c2.wav',
 '/content/train/farfield/566e4f7d788d755e2fa76ab57837d7dd.wav',
 '/content/train/farfield/40bf19adace8d787cd3c63784ed0b27b.wav']

In [30]:
texts[:10]

['афина хотела номер телефона поменять карты',
 'салют хочу поговорить с джой',
 'джой какая валюта в камбодже',
 'салют я могу что нибудь сделать',
 'джой смотреть операцию ы и другие приключения шурика онлайн',
 'салют на каком языке создана кавка на пляже',
 'сбер чем платить на канарах',
 'афина на пару минут перемотай вперед',
 'афина площадь королевства тонга',
 'афина что такое делфт']

In [31]:
transcriptions[:10]

['афина хотела номер телефона поменять карты',
 'салют хочу поговорить с джой',
 'джой какая валюта в камбодже',
 'салют я могу что нибудь сделать',
 'джой смотреть операцию ы и другие приключения шурика онлайн',
 'салют на каком языке создана кавка на пляже',
 'сбер чем платить на канарах',
 'афина на пару минут перемотай вперед',
 'афина площадь королевства тонга',
 'афина что такое делфт']

## Обзор ошибок

In [32]:
!pip install jiwer
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149867 sha256=d884e25db536b596d9a636650513b1ce6dd70ebd5922a79573b43b54d495fd81
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [33]:
from jiwer import wer
from Levenshtein import ratio

In [34]:
len(texts), len(files)

(123999, 123999)

In [35]:
import nltk
WER = []
LEV = []
RAT = []
# SUB = []
# INS = []
# DEL = []
for i in range(len(texts)):
  try:
    WER.append(wer(texts[i], transcriptions[i]))
    LEV.append(nltk.edit_distance(texts[i], transcriptions[i]))
    RAT.append(ratio(texts[i], transcriptions[i]))
  except:
    WER.append(None)
    LEV.append(nltk.edit_distance(texts[i], transcriptions[i]))
    RAT.append(ratio(texts[i], transcriptions[i]))

In [36]:
texts[:5], transcriptions[:5], files[:5]

(['афина хотела номер телефона поменять карты',
  'салют хочу поговорить с джой',
  'джой какая валюта в камбодже',
  'салют я могу что нибудь сделать',
  'джой смотреть операцию ы и другие приключения шурика онлайн'],
 ['афина хотела номер телефона поменять карты',
  'салют хочу поговорить с джой',
  'джой какая валюта в камбодже',
  'салют я могу что нибудь сделать',
  'джой смотреть операцию ы и другие приключения шурика онлайн'],
 ['/content/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav',
  '/content/train/farfield/f6075fae0c592183f107cf781117d9b5.wav',
  '/content/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav',
  '/content/train/farfield/5f7483035b2f26b773b0d3fb125b779d.wav',
  '/content/train/farfield/4c95a12b71e1262ac937d2fe6a803906.wav'])

In [37]:
import pandas as pd

results = pd.DataFrame()
results['id'] = ids
results['true'] = texts
results['pred'] = transcriptions
results['file'] = files
results['WER'] = WER
results['LEV'] = LEV
results['RAT'] = RAT

# test_results['SUB'] = SUB
# test_results['INS'] = INS
# test_results['DEL'] = DEL

In [42]:
results[results['id'] == 'd24e697c7c381de1f0ba49bf6daf2da7'].file.values

array(['/content/train/farfield/d24e697c7c381de1f0ba49bf6daf2da7.wav'],
      dtype=object)

In [43]:
results.loc[107923].file

'/content/train/farfield/ef104f705011ef2d10f4403dd9dc09e3.wav'

In [44]:
asr_model.transcribe(['/content/train/farfield/ef104f705011ef2d10f4403dd9dc09e3.wav'])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-11-30 06:23:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


['нвракилова на уинс']

In [45]:
# Load and listen to the audio file
example_file = '/content/train/farfield/ef104f705011ef2d10f4403dd9dc09e3.wav'
audio, sample_rate = librosa.load(example_file)
ipd.Audio(example_file, rate=sample_rate)

## Pytorch trainer

In [50]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [51]:
params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'

params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size']=16

params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size']=16

params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size']=16

params_model['optim']['lr'] = 0.00015

In [52]:
from omegaconf import DictConfig

first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params_model), trainer=trainer)

[NeMo I 2021-11-30 06:24:05 collections:173] Dataset loaded with 40807 files totalling 43.58 hours
[NeMo I 2021-11-30 06:24:05 collections:174] 1 files were filtered totalling 0.01 hours


[NeMo W 2021-11-30 06:24:05 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 06:24:05 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 06:24:05 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 06:24:05 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 06:24:05 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 06:24:05 features:265] PADDING: 16
[NeMo I 2021-11-30 06:24:05 features:282] STFT using torch


In [53]:
asr_model.setup_training_data(train_data_config=params_model['train_ds'])
asr_model.setup_test_data(test_data_config=params_model['test_ds'])
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo I 2021-11-30 06:24:09 collections:173] Dataset loaded with 40807 files totalling 43.58 hours
[NeMo I 2021-11-30 06:24:09 collections:174] 1 files were filtered totalling 0.01 hours


[NeMo W 2021-11-30 06:24:09 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 06:24:09 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 06:24:09 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 06:24:09 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 06:24:09 collections:174] 0 files were filtered totalling 0.00 hours


In [54]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 06:24:10 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.6706365164969896


In [55]:
trainer.fit(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 06:24:58 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 06:24:58 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.00015
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 06:24:58 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 06:24:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 06:25:01 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 06:54:32 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    
[NeMo W 2021-11-30 06:54:35 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for

Validating: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 07:41:59 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


In [56]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 07:42:43 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.59790523876723471
